In [2]:
import pandas as pd
df = pd.read_csv('/content/Constraint_Train.csv', encoding='ISO-8859-1')
df = df.head(100)
train_texts=list(df['tweet'])
train_labels=list(df['label'])
train_labels=list(pd.get_dummies(train_labels,drop_first=True)['real'])

df = pd.read_csv('/content/Constraint_Val.csv', encoding='ISO-8859-1')
df = df.head(30)
val_texts=list(df['tweet'])
val_labels=list(df['label'])
val_labels=list(pd.get_dummies(val_labels,drop_first=True)['real'])

df = pd.read_csv('/content/Constraint_test.csv', encoding='ISO-8859-1')
df = df.head(30)
test_texts=list(df['tweet'])
test_labels=list(df['label'])
test_labels=list(pd.get_dummies(test_labels,drop_first=True)['real'])
# df.shape


# train_labels

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# !pip install transformers[torch]

# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

from transformers import DistilBertTokenizerFast
# tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# from transformers import DistilBertTokenizerFast
# tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

# print(len(val_texts))
# print(val_texts)
# print(type(val_encodings))
# print(val_encodings)

import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

# train_dataset[5]

from transformers import Trainer, TrainingArguments, BertForSequenceClassification

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    gradient_accumulation_steps=1
)

model = AutoModelForSequenceClassification.from_pretrained("roberta-base")
# model = BertForSequenceClassification.from_pretrained("sarkerlab/SocBERT-base")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

trainer.evaluate(test_dataset)



trainer.predict(test_dataset)

trainer.predict(test_dataset)[1].shape

output=trainer.predict(test_dataset)[1]

from sklearn.metrics import confusion_matrix

cm=confusion_matrix(test_labels, output)
# cm

trainer.save_model('RoBERTa_model')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.754300
20,0.745900
30,0.773500
40,0.672300
50,0.692400
60,0.759700
70,0.679700
80,0.713000
90,0.663700
100,0.676400


In [14]:
import pandas as pd
df = pd.read_csv('/content/Constraint_Train.csv', encoding='ISO-8859-1')
df = df.head(100)
train_texts=list(df['tweet'])
train_labels=list(df['label'])

df = pd.read_csv('/content/Constraint_Val.csv', encoding='ISO-8859-1')
df = df.head(30)
val_texts=list(df['tweet'])
val_labels=list(df['label'])

df = pd.read_csv('/content/Constraint_test.csv', encoding='ISO-8859-1')
df = df.head(30)
test_texts=list(df['tweet'])
test_labels=list(df['label'])
# df.shape

In [15]:
train_labels=list(pd.get_dummies(train_labels,drop_first=True)['real'])
val_labels=list(pd.get_dummies(val_labels,drop_first=True)['real'])
test_labels=list(pd.get_dummies(test_labels,drop_first=True)['real'])

In [ ]:
# train_labels

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [1]:
!pip install transformers[torch]

In [17]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

from transformers import DistilBertTokenizerFast
# tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
# from transformers import DistilBertTokenizerFast
# tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [18]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

In [ ]:
print(len(val_texts))
print(val_texts)
print(type(val_encodings))
print(val_encodings)

30
['Chinese converting to Islam after realising that no muslim was affected by #Coronavirus #COVD19 in the country', '11 out of 13 people (from the Diamond Princess Cruise ship) who had intially tested negative in tests in Japan were later confirmed to be positive in the United States.', 'COVID-19 Is Caused By A Bacterium, Not Virus And Can Be Treated With Aspirin', "Mike Pence in RNC speech praises Donald Trumpâ\x80\x99s COVID-19 â\x80\x9cseamlessâ\x80\x9d partnership with governors and leaves out the president's state feuds: https://t.co/qJ6hSewtgB #RNC2020 https://t.co/OFoeRZDfyY", "6/10 Sky's @EdConwaySky explains the latest #COVID19 data and government announcement. Get more on the #coronavirus data hereð\x9f\x91\x87 https://t.co/jvGZlSbFjH https://t.co/PygSKXesBg", 'No one can leave managed isolation for any reason without returning a negative test. If they refuse a test they can then be held for a period of up to 28 days. â\x81£ â\x81£ On June the 16th exemptions on compassiona

In [19]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [ ]:
train_dataset[5]

In [20]:
from transformers import Trainer, TrainingArguments, BertForSequenceClassification

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1
)

In [21]:
model = AutoModelForSequenceClassification.from_pretrained("roberta-base")
# model = BertForSequenceClassification.from_pretrained("sarkerlab/SocBERT-base")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
trainer.train()

Step,Training Loss
1,0.775300
2,0.600700
3,0.792500
4,0.763500
5,0.735200
6,0.763000
7,0.500700
8,0.559400
9,0.540900
10,0.499900


TrainOutput(global_step=100, training_loss=1.0268887466657908, metrics={'train_runtime': 12.8194, 'train_samples_per_second': 7.801, 'train_steps_per_second': 7.801, 'total_flos': 7554165066000.0, 'train_loss': 1.0268887466657908, 'epoch': 1.0})

In [23]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.6234884858131409,
 'eval_runtime': 1.1327,
 'eval_samples_per_second': 26.486,
 'eval_steps_per_second': 26.486,
 'epoch': 1.0}

In [24]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-2.5559509 ,  2.7681262 ],
       [ 0.04195629, -0.08116461],
       [ 0.0421141 ,  0.04737667],
       [-2.324264  ,  2.613964  ],
       [-0.88124394,  1.0040797 ],
       [-0.94549954,  1.0451622 ],
       [-1.8400902 ,  2.1112416 ],
       [-0.11342328,  0.16150379],
       [-2.6947258 ,  2.8709815 ],
       [-2.2441635 ,  2.4187236 ],
       [-1.1391364 ,  1.3310229 ],
       [ 0.06187572, -0.04798014],
       [-1.8842508 ,  2.0843213 ],
       [-0.32520664,  0.4541872 ],
       [ 0.10788678, -0.08128266],
       [-0.9861113 ,  1.1583264 ],
       [-0.32898915,  0.38364446],
       [ 0.01330426,  0.0626809 ],
       [-2.6568034 ,  2.8255754 ],
       [-1.4920697 ,  1.7068912 ],
       [-0.96929836,  1.1031146 ],
       [ 0.16256465, -0.12915666],
       [-1.8373505 ,  2.0207405 ],
       [-0.2926962 ,  0.3299022 ],
       [-1.7737709 ,  1.9972073 ],
       [-1.4281087 ,  1.5330293 ],
       [-0.03927647,  0.04861167],
       [-0.9862964 ,  1.06

In [27]:
trainer.predict(test_dataset)[1].shape

(30,)

In [28]:
output=trainer.predict(test_dataset)[1]

In [29]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(test_labels, output)
cm

array([[15,  0],
       [ 0, 15]])

In [30]:
trainer.save_model('senti_model')